# An Overview of the TNMCMC for 3D Spin Glass Simulation Algorithm

This notebook provides a concise explanation of the simulation workflow for studying the 3D Edwards-Anderson spin glass model, as implemented in `3Drunargs.jl` and its associated modules.

## 1. Project Goal & Methodology

The primary goal of this project is to simulate the **3D Edwards-Anderson (EA) spin glass model** with open boundary conditions. A major challenge in simulating spin glasses, especially at low temperatures, is **critical slowing down**, where traditional local-update Monte Carlo methods (like single-spin flips) become extremely inefficient at exploring the system's complex energy landscape.

To overcome this, the project implements two distinct algorithms:
1.  A standard **Metropolis Monte Carlo** algorithm as a baseline.
2.  A more advanced **Tensor Network Metropolis-Hastings (TNMH)** algorithm, which performs non-local, collective updates on large blocks of spins to accelerate convergence.

## 2. How to Run the Simulation

The simulation is launched from the command line via the main script `3Drunargs.jl`. The behavior of the simulation is controlled by a rich set of command-line arguments.

### Example Command:
```bash
julia 3Drunargs.jl  --Type FBC --P 0.5 --zchunk 1 --ychunk 4 --xchunk 4 --Lz 4 --L 4 --Dimension 3 --Beta 1.5 --chi 4 --Ndisorder 10 --Nsample 100 --Ntherm 100 --step 1 --Nreplic 2 --seed 1

In [1]:
include("../src/SpinGlass.jl")
using .SpinGlass
using Random
using FileIO

In [4]:
# 1. input
judge = "FBC"
Beta = 1.5
P = 0.5
L = 4
Lz = 4
zchunk = 1
ychunk = 4
xchunk = 4
Dimension = 3
chi = 4
Ndisorder = 1
Nsample = 10
Ntherm = 10
step = 0
Nreplic = 2
seed = 1
verbose = 0
T = [Beta,P]
parsed_params = [L, Lz,zchunk, ychunk, xchunk, Dimension, chi, Ndisorder, Nsample, Ntherm, step, Nreplic, seed, verbose]

# 2. Initialize data structures for parameters, spin configurations, statistics, etc.
pa = Parament(judge, T..., parsed_params...)
sp = Spin(pa)
st = Statistics(pa)
da = DataFile(pa)
te = Tensor(pa.chi,pa.zchunk,pa.ychunk,pa.xchunk)
Random.seed!(parsed_params[end])

# 3. Seed the random number generator for reproducibility
Random.seed!(parsed_params[end])

SimulationTNMH(pa, sp, st, da) # Use Tensor Network Method

Elapsed time (seconds): 0.174


## 3. Deep Dive into the TNMH Algorithm

The TNMH algorithm is a method designed to perform efficient, non-local updates. Instead of flipping one spin at a time, it proposes a collective update for a large 3D block of spins.

### TNMH Workflow Step-by-Step

The `SimulationTNMH` function follows a structured, multi-layered loop.

**1. Disorder Loop:**
The outermost loop iterates from `idis in 1:pa.Ndisorder`, just like the standard Metropolis. This ensures results are averaged over many different random spin glass instances.

**2. Thermalization and Sampling Loops:**
Inside the disorder loop, there are standard `itherm` and `isamp` loops for thermalization and measurement. The key difference is what happens inside each simulation step.

**3. The "Sweep":**
A single "sweep" in this algorithm is highly sophisticated and consists of three main parts:
    
   **a. Directional TNMH Block Updates:**
   The code iterates through a set of directions (e.g., `:zyx2yxz`, etc.). This ensures that the blocks being updated sweep through the 3D lattice in different ways to avoid bias. For each direction, the algorithm does the following:
   * It iterates through the entire 3D lattice, selecting **overlapping 3D blocks** of spins of size `zchunk x ychunk x xchunk`.
   * **Tensor Network Construction (`InitTensor`)**: For a selected block, it builds a local tensor network representation. The bond dimension `chi` is used here to compress the environment information, making the calculation feasible.
   * **Block Sampling (`TNMH_oe`)**: It then samples a **brand-new configuration for the entire block** from the probability distribution described by the tensor network. This is a massive, collective move that can cross energy barriers that would be impossible for single-spin flips.
   * **Metropolis-Hastings Acceptance (`Accept`)**: The proposed update for the entire block is then accepted or rejected based on the standard Metropolis-Hastings criterion.

This entire three-part process constitutes one step in the `itherm` or `isamp` loop.

### Simplified TNMH Loop Structure

Here is a conceptual representation of the core loop in `SimulationTNMH`:

```julia
# Simplified pseudo-code for the main simulation loop

for idis in 1:Ndisorder          # Loop over random bond configurations
    # ... Load or generate bonds ...
    
    for itherm in 1:Ntherm       # Thermalization loop
        for direction in [:zyx, :yxz, :xzy]  # Directional sweeps
        
            # A: Perform TNMH block updates across the entire lattice
            for block in all_blocks_in_lattice
                propose_new_state_for_block_using_tensors()
                accept_or_reject_block_update()
            end

            # B: Perform a quick global Metropolis sweep
            perform_single_spin_flips()
        end
        # C: Perform a global spin flip for symmetry
        flip_all_spins()
    end
    
    for isamp in 1:Nsample         # Measurement loop (repeats the same hybrid sweep)
        # ... (same sweep logic as above) ...
        measure_observables()
    end
    
    normalize_and_save_results()
end